# Otras técnicas soportadas por RAPIDS

In [ ]:
import os
import os.path

import cudf
import cupy as cp
import cuml
from cuml.linear_model import LogisticRegression, MBSGDClassifier, MBSGDRegressor
from cuml.multiclass import MulticlassClassifier
from cuml.naive_bayes import MultinomialNB
from cuml.ensemble import RandomForestClassifier
from cuml.svm import SVC
from cuml.metrics.regression import mean_squared_error as mnsq

%run ../utils/f_northing.py
%run ../utils/f_northing_numpy.py
%run ../utils/f_price_range.py
%run ../utils/f_static_data.py
%run ../utils/f_utils.py

cities_to_use = ['sevilla']
#cities_to_use = ['shanghai']
#cities_to_use = cities_to_use_1()
#cities_to_use = cities_to_use_2()
columns_to_use = columns_to_use()
columns_to_fit = columns_to_fit()

cuml.set_global_output_type('cudf')

In [ ]:
listings = cudf.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df = cudf.read_csv(directory + file, usecols = columns_to_use)
                standard_object_type(temp_df, ['host_acceptance_rate', 'neighbourhood_cleansed'])
                if(temp_df['host_total_listings_count'].dtype != 'float32'):
                    temp_df['host_total_listings_count'] = temp_df['host_total_listings_count'].fillna(-1).astype('float32')
                if(temp_df['bathrooms'].dtype != 'float32'):
                    temp_df['bathrooms'] = temp_df['bathrooms'].fillna(-1).astype('float32')
                if(temp_df['bedrooms'].dtype != 'float32'):
                    temp_df['bedrooms'] = temp_df['bedrooms'].fillna(-1).astype('float32')
                if(temp_df['beds'].dtype != 'float32'):
                    temp_df['beds'] = temp_df['beds'].fillna(-1).astype('float32')
                if listings.size == 0:
                    listings = temp_df
                else:
                    for column in listings.columns:
                        if listings[column].dtype != temp_df[column].dtype:
                            print('Found error: '+column+' type '+listings[column].dtype.name+' doesnt match '+temp_df[column].dtype.name)
                    listings = listings.append(temp_df)
                    
listings = listings.drop_duplicates()
listings = listings.reset_index(drop=True)

type_conversion(listings, ['host_id', 'accommodates', 'number_of_reviews', 'reviews_per_month', 'minimum_nights', 'maximum_nights', 'availability_30', 'availability_90', 'availability_365', 'number_of_reviews_ltm', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'host_total_listings_count', 'bathrooms', 'bedrooms', 'beds'])
column_factorize(listings, ['neighbourhood_cleansed', 'host_response_time', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'property_type', 'room_type', 'instant_bookable'])

clean_format_strings(listings, ['host_response_rate', 'host_acceptance_rate'])
clean_format_price(listings, ['price'])
listings['price'] = listings['price'].applymap(priceRange, 'float32')
listings['price'] = listings['price'].astype('float32')

cupy_lat = cp.asarray(listings['latitude'])
cupy_long = cp.asarray(listings['longitude'])
n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
listings['northing'] = cudf.Series(n_cupy_array).astype('float32')
listings['easting'] = cudf.Series(e_cupy_array).astype('float32')

listings.head()

## RandomForest

Parece ser que la operación sort de cupy (basada en numpy.sort) tiene un límite estricto de elementos, ya sea por tamaño o por memoria disponible.

Tomamos una sección del dataset con exactamente 5405 filas y ejecutamos un clasificador RandomForest multiclase:

In [ ]:
listings_section = listings[0:5405]

cuml_model = RandomForestClassifier(max_features=1.0,
                   n_bins=8,
                   n_estimators=40)

x_train, x_test, y_train, y_test  = cuml.train_test_split(listings_section[columns_to_fit], listings_section['price'], train_size=0.9)

cuml_model.fit(x_train,y_train)
cuml_predict = cuml_model.predict(x_test)

print("Predicted labels : ", cuml_predict[0:10])
print("Real : ", y_test[0:10])

Ejecutamos el mismo clasificador con una fila más:

In [ ]:
listings_section = listings[0:5406]

x_train, x_test, y_train, y_test  = cuml.train_test_split(listings_section[columns_to_fit], listings_section['price'], train_size=0.9)

cuml_model.fit(x_train,y_train)
cuml_predict = cuml_model.predict(x_test)

print("Predicted labels : ", cuml_predict[0:10])
print("Real : ", y_test[0:10])

Es posible ver una traza de error más detallada si, en lugar de ejecutar el caso anterior, se ejecuta el siguiente caso con un dataset generado aleatoriamente:

In [ ]:
X = cp.random.normal(size=(5406,4)).astype(np.float32)
y = cp.random.randint(2,size=(5406,1)).astype(np.int32)

cuml_model = RandomForestClassifier(max_features=1.0,
                   n_bins=8,
                   n_estimators=40)

x_train, x_test, y_train, y_test  = cuml.train_test_split(X, y, train_size=0.9)

cuml_model.fit(x_train,y_train)
cuml_predict = cuml_model.predict(x_test)

print("Predicted labels : ", cuml_predict[0:10])
print("Real : ", y_test[0:10])

Este resultado es dependiente de la memoria disponible en el sistema: los ejemplos anteriores se han realizado sobre una tarjeta gráfica NVIDIA GTX 1060 con 6GB de VRAM, mientras que en una GPU Tesla V100 con 16GB de VRAM no se producen errores.

## Clasificación/Regresión mediante SGD/MiniBatch SGD (Stochastic Gradient Descent)

Los algoritmos de esta familia producen resultados varios según se traten de regresión o clasificación.

El clasificador MiniBatch produce errores de memoria o es incapaz de ejecutarse completamente:

In [ ]:
model = MBSGDClassifier(learning_rate='constant', eta0=0.05, epochs=2000, fit_intercept=True, batch_size=1, tol=0.0, penalty='l2', loss='squared_loss', alpha=0.5) #todo 0
x_train, x_test, y_train, y_test  = cuml.train_test_split(listings[columns_to_fit], listings['price'], train_size=0.9)
x_test_index = x_test.reset_index(drop=True)
y_test_index = y_test.reset_index(drop=True)
model.fit(x_train, y_train)

In [ ]:
predictions = model.predict(x_test_index)
y_results = cudf.DataFrame()
y_results['prediction'] = predictions
y_results['real'] = y_test_index
y_results[0:10]

El regresor MiniBatch tarda demasiado en ejecutarse, con lo cual no es posible determinar si ha dejado de funcionar debido a falta de memoria.

In [ ]:
model = MBSGDRegressor(learning_rate='constant', eta0=0.05, epochs=2000, fit_intercept=True, batch_size=1, tol=0.0, penalty='l2', loss='squared_loss', alpha=0.5)
x_train, x_test, y_train, y_test  = cuml.train_test_split(listings[columns_to_fit], listings['price'], train_size=0.9)
x_test_index = x_test.reset_index(drop=True)
y_test_index = y_test.reset_index(drop=True)
model.fit(x_train, y_train)

In [ ]:
predictions = model.predict(x_test_index)
y_results = cudf.DataFrame()
y_results['prediction'] = predictions
y_results['real'] = y_test_index
y_results[0:10]

De manera similar, el regresor por descenso de gradiente normal también se queda parado
durante la ejecución.

In [ ]:
model = cuml.SGD(learning_rate='constant', eta0=0.005, epochs=2000, fit_intercept=True, batch_size=2,tol=0.0, penalty='none', loss='squared_loss')
x_train, x_test, y_train, y_test  = cuml.train_test_split(listings[columns_to_fit], listings['price'], train_size=0.9)
x_test_index = x_test.reset_index(drop=True)
y_test_index = y_test.reset_index(drop=True)
model.fit(x_train, y_train)

In [ ]:
predictions = model.predict(x_test_index)
y_results = cudf.DataFrame()
y_results['prediction'] = predictions
y_results['real'] = y_test_index
y_results[0:10]

## Clasificador multiclase

- En datasets de cierto tamaño, devuelve el mismo error de tamaño que ya encontramos en otros clasificadores.
- En datasets pequeños, devuelve errores 'L-BFGS line search failed' - posible error de memoria o de entrada de datos.
- No ha sido posible probar este algoritmo en el entorno de Tesla V100, debido a que este algoritmo aún no estaba implementado en la versión de RAPIDS empleada allí.

In [ ]:
model = MulticlassClassifier(LogisticRegression(), strategy='ovo')
x_train, x_test, y_train, y_test  = cuml.train_test_split(listings[columns_to_fit], listings['price'], train_size=0.9)
x_test_index = x_test.reset_index(drop=True)
y_test_index = y_test.reset_index(drop=True)
model.fit(x_train, y_train)

In [ ]:
predictions = model.predict(x_test_index)
y_results = cudf.DataFrame()
y_results['prediction'] = predictions
y_results['real'] = y_test_index
y_results[0:10]

## Naive Bayes multinomial

Devuelve el mismo error de sort que los clasificadores.

In [ ]:
model = MultinomialNB()
x_train, x_test, y_train, y_test  = cuml.train_test_split(listings[columns_to_fit], listings['price'], train_size=0.9)
x_test_index = x_test.reset_index(drop=True)
y_test_index = y_test.reset_index(drop=True)
model.fit(x_train, y_train)

In [ ]:
predictions = model.predict(x_test_index)
y_results = cudf.DataFrame()
y_results['prediction'] = predictions
y_results['real'] = y_test_index
y_results[0:10]

En el entorno Tesla V100, usando una versión deprecada de RAPIDS, se obtiene un error de compilación:

CompileException: /tmp/tmpgu_vlsfv/9f7780251c2102e5378a204d4a270d95_2.cubin.cu(25): error: expression must have integral or enum type

1 error detected in the compilation of "/tmp/tmpgu_vlsfv/9f7780251c2102e5378a204d4a270d95_2.cubin.cu".

## Regresión mediante algoritmos Quasi-Newton

- En el entorno GTX 1060. devuelve el mismo error de sort que los clasificadores.
- En el entorno Tesla V100, se ejecuta correctamente, con lo que puede deberse a un error de memoria.

In [ ]:
model = cuml.QN(loss='softmax')
x_train, x_test, y_train, y_test  = cuml.train_test_split(listings[columns_to_fit], listings['price'], train_size=0.9)
x_test_index = x_test.reset_index(drop=True)
y_test_index = y_test.reset_index(drop=True)
model.fit(x_train, y_train)

In [ ]:
predictions = model.predict(x_test_index)
y_results = cudf.DataFrame()
y_results['prediction'] = predictions
y_results['real'] = y_test_index
y_results[0:10]

## Clasificación mediante soporte de vectores SVC (C-Support Vector Classification)

Devuelve el mismo error de sort que los clasificadores. Adicionalmente, sólo soporta clasificación binaria, con lo cual no es usable en el ejemplo de este proyecto.

In [ ]:
model = SVC(kernel='poly', degree=2, gamma='auto', C=1)
x_train, x_test, y_train, y_test  = cuml.train_test_split(listings[columns_to_fit], listings['price'], train_size=0.9)
x_test_index = x_test.reset_index(drop=True)
y_test_index = y_test.reset_index(drop=True)
model.fit(x_train, y_train)

In [ ]:
predictions = model.predict(x_test_index)
y_results = cudf.DataFrame()
y_results['prediction'] = predictions
y_results['real'] = y_test_index
y_results[0:10]